# Week 16 - Day 2: Multi-Armed Bandits

## Learning Objectives
- Understand the exploration vs exploitation tradeoff
- Implement Epsilon-Greedy strategy
- Implement Thompson Sampling with Beta distributions
- Implement Upper Confidence Bound (UCB) algorithm
- Apply bandits to trading strategy selection

## The Multi-Armed Bandit Problem

The multi-armed bandit (MAB) problem is a classic reinforcement learning problem that models the exploration-exploitation tradeoff:

- **Setting**: You face K slot machines ("arms"), each with an unknown reward probability
- **Goal**: Maximize cumulative reward over T time steps
- **Challenge**: Balance exploring new arms vs exploiting known good arms

### Finance Applications
- **Strategy Selection**: Choose among multiple trading strategies
- **Portfolio Allocation**: Dynamic allocation across assets
- **A/B Testing**: Test different execution algorithms
- **Market Making**: Adaptive spread setting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
from dataclasses import dataclass
from abc import ABC, abstractmethod

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

Libraries loaded successfully!


---
## Part 1: Bandit Environment

First, let's create a simulated bandit environment where each arm represents a trading strategy with different success probabilities.

In [2]:
class BanditEnvironment:
    """
    Multi-Armed Bandit environment simulating trading strategies.
    
    Each arm represents a strategy with:
    - A true (hidden) probability of generating positive returns
    - Bernoulli rewards: 1 (profitable trade) or 0 (losing trade)
    """
    
    def __init__(self, n_arms: int, true_probabilities: List[float] = None):
        """
        Initialize the bandit environment.
        
        Args:
            n_arms: Number of arms (strategies)
            true_probabilities: True win rates for each strategy
        """
        self.n_arms = n_arms
        
        if true_probabilities is None:
            # Generate random probabilities
            self.true_probabilities = np.random.uniform(0.3, 0.7, n_arms)
        else:
            self.true_probabilities = np.array(true_probabilities)
        
        self.best_arm = np.argmax(self.true_probabilities)
        self.best_probability = self.true_probabilities[self.best_arm]
        
    def pull(self, arm: int) -> int:
        """Pull an arm and get reward (0 or 1)."""
        return int(np.random.random() < self.true_probabilities[arm])
    
    def get_regret(self, arm: int) -> float:
        """Calculate regret for choosing a suboptimal arm."""
        return self.best_probability - self.true_probabilities[arm]
    
    def __repr__(self):
        return f"BanditEnvironment(n_arms={self.n_arms}, best_arm={self.best_arm}, best_prob={self.best_probability:.3f})"


# Create environment with 5 trading strategies
strategy_names = ['Momentum', 'Mean Reversion', 'Breakout', 'Pairs Trading', 'Trend Following']
true_win_rates = [0.52, 0.48, 0.55, 0.51, 0.58]  # Realistic win rates

env = BanditEnvironment(n_arms=5, true_probabilities=true_win_rates)
print(f"Environment: {env}")
print(f"\nStrategy Win Rates (hidden from algorithms):")
for i, (name, prob) in enumerate(zip(strategy_names, true_win_rates)):
    best_marker = " ← BEST" if i == env.best_arm else ""
    print(f"  {name}: {prob:.1%}{best_marker}")

Environment: BanditEnvironment(n_arms=5, best_arm=4, best_prob=0.580)

Strategy Win Rates (hidden from algorithms):
  Momentum: 52.0%
  Mean Reversion: 48.0%
  Breakout: 55.0%
  Pairs Trading: 51.0%
  Trend Following: 58.0% ← BEST


---
## Part 2: Epsilon-Greedy Strategy

The simplest exploration strategy:
- With probability ε: **explore** (choose random arm)
- With probability 1-ε: **exploit** (choose best known arm)

### Mathematical Formulation

$$
a_t = \begin{cases}
\text{random arm} & \text{with probability } \epsilon \\
\arg\max_a \hat{Q}(a) & \text{with probability } 1 - \epsilon
\end{cases}
$$

Where $\hat{Q}(a)$ is the estimated value of arm $a$:

$$\hat{Q}(a) = \frac{\text{Total rewards from arm } a}{\text{Number of times arm } a \text{ was pulled}}$$

In [3]:
class BanditAlgorithm(ABC):
    """Abstract base class for bandit algorithms."""
    
    def __init__(self, n_arms: int):
        self.n_arms = n_arms
        self.reset()
    
    def reset(self):
        """Reset algorithm state."""
        self.counts = np.zeros(self.n_arms)  # Number of times each arm was pulled
        self.values = np.zeros(self.n_arms)  # Estimated value of each arm
        self.total_reward = 0
        self.t = 0
    
    @abstractmethod
    def select_arm(self) -> int:
        """Select which arm to pull."""
        pass
    
    def update(self, arm: int, reward: float):
        """Update estimates after receiving reward."""
        self.t += 1
        self.counts[arm] += 1
        self.total_reward += reward
        
        # Incremental mean update
        n = self.counts[arm]
        self.values[arm] += (reward - self.values[arm]) / n


class EpsilonGreedy(BanditAlgorithm):
    """
    Epsilon-Greedy Algorithm.
    
    Explores with probability epsilon, exploits otherwise.
    """
    
    def __init__(self, n_arms: int, epsilon: float = 0.1):
        super().__init__(n_arms)
        self.epsilon = epsilon
    
    def select_arm(self) -> int:
        """Select arm using epsilon-greedy policy."""
        if np.random.random() < self.epsilon:
            # Explore: choose random arm
            return np.random.randint(self.n_arms)
        else:
            # Exploit: choose best known arm (break ties randomly)
            max_value = np.max(self.values)
            best_arms = np.where(self.values == max_value)[0]
            return np.random.choice(best_arms)
    
    def __repr__(self):
        return f"EpsilonGreedy(ε={self.epsilon})"


class DecayingEpsilonGreedy(BanditAlgorithm):
    """
    Epsilon-Greedy with decaying epsilon.
    
    Epsilon decays over time: ε_t = ε_0 / (1 + decay * t)
    This reduces exploration as we learn more.
    """
    
    def __init__(self, n_arms: int, epsilon_start: float = 1.0, decay: float = 0.01):
        super().__init__(n_arms)
        self.epsilon_start = epsilon_start
        self.decay = decay
    
    @property
    def epsilon(self) -> float:
        """Current epsilon value."""
        return self.epsilon_start / (1 + self.decay * self.t)
    
    def select_arm(self) -> int:
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_arms)
        else:
            max_value = np.max(self.values)
            best_arms = np.where(self.values == max_value)[0]
            return np.random.choice(best_arms)
    
    def __repr__(self):
        return f"DecayingEpsilonGreedy(ε₀={self.epsilon_start}, decay={self.decay})"


# Test epsilon-greedy
eg = EpsilonGreedy(n_arms=5, epsilon=0.1)
print(f"Algorithm: {eg}")
print(f"Initial values: {eg.values}")

Algorithm: EpsilonGreedy(ε=0.1)
Initial values: [0. 0. 0. 0. 0.]


---
## Part 3: Thompson Sampling

Thompson Sampling is a **Bayesian** approach that maintains probability distributions over arm values.

### Algorithm
1. Maintain Beta(α, β) distribution for each arm
2. Sample from each distribution
3. Select arm with highest sample
4. Update distribution with observed reward

### Beta Distribution for Bernoulli Bandits

For Bernoulli rewards (0 or 1):
- Prior: Beta(1, 1) = Uniform(0, 1)
- After s successes and f failures: Beta(1 + s, 1 + f)

### Why Thompson Sampling Works
- Arms with **uncertainty** have wider distributions → more likely to be sampled
- Arms with **high mean** are more likely to have high samples
- Naturally balances exploration and exploitation

In [4]:
class ThompsonSampling(BanditAlgorithm):
    """
    Thompson Sampling for Bernoulli bandits.
    
    Uses Beta distributions as conjugate priors for Bernoulli likelihood.
    """
    
    def __init__(self, n_arms: int, prior_alpha: float = 1.0, prior_beta: float = 1.0):
        super().__init__(n_arms)
        self.prior_alpha = prior_alpha
        self.prior_beta = prior_beta
        
    def reset(self):
        super().reset()
        # Beta distribution parameters for each arm
        self.alpha = np.ones(self.n_arms) * self.prior_alpha  # successes + 1
        self.beta = np.ones(self.n_arms) * self.prior_beta    # failures + 1
    
    def select_arm(self) -> int:
        """Sample from each arm's Beta distribution and select highest."""
        # Sample theta from Beta(alpha, beta) for each arm
        samples = np.random.beta(self.alpha, self.beta)
        return np.argmax(samples)
    
    def update(self, arm: int, reward: float):
        """Update Beta distribution parameters."""
        super().update(arm, reward)
        
        # Update Beta parameters
        if reward == 1:
            self.alpha[arm] += 1
        else:
            self.beta[arm] += 1
    
    def get_expected_values(self) -> np.ndarray:
        """Get expected value (mean of Beta distribution) for each arm."""
        return self.alpha / (self.alpha + self.beta)
    
    def get_uncertainty(self) -> np.ndarray:
        """Get uncertainty (std of Beta distribution) for each arm."""
        a, b = self.alpha, self.beta
        return np.sqrt((a * b) / ((a + b)**2 * (a + b + 1)))
    
    def __repr__(self):
        return f"ThompsonSampling(prior=Beta({self.prior_alpha}, {self.prior_beta}))"


# Visualize Thompson Sampling distributions
def visualize_thompson_distributions(ts: ThompsonSampling, strategy_names: List[str]):
    """Visualize Beta distributions for each arm."""
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.linspace(0, 1, 200)
    colors = plt.cm.tab10(np.linspace(0, 1, ts.n_arms))
    
    for i in range(ts.n_arms):
        from scipy import stats
        y = stats.beta.pdf(x, ts.alpha[i], ts.beta[i])
        ax.plot(x, y, label=f"{strategy_names[i]} (α={ts.alpha[i]:.0f}, β={ts.beta[i]:.0f})",
                color=colors[i], linewidth=2)
        ax.fill_between(x, y, alpha=0.2, color=colors[i])
    
    ax.set_xlabel('Win Probability')
    ax.set_ylabel('Probability Density')
    ax.set_title('Thompson Sampling: Posterior Distributions')
    ax.legend(loc='upper right')
    ax.set_xlim(0, 1)
    
    plt.tight_layout()
    plt.show()


# Test Thompson Sampling
ts = ThompsonSampling(n_arms=5)
print(f"Algorithm: {ts}")
print(f"Initial alpha: {ts.alpha}")
print(f"Initial beta: {ts.beta}")

AttributeError: 'ThompsonSampling' object has no attribute 'prior_alpha'

---
## Part 4: Upper Confidence Bound (UCB)

UCB follows the principle of **"optimism in the face of uncertainty"**:
- Choose the arm with highest upper confidence bound
- UCB = estimated mean + exploration bonus

### UCB1 Formula

$$a_t = \arg\max_a \left[ \hat{Q}(a) + c \cdot \sqrt{\frac{\ln t}{N(a)}} \right]$$

Where:
- $\hat{Q}(a)$ = estimated value of arm $a$
- $t$ = total number of rounds
- $N(a)$ = number of times arm $a$ was pulled
- $c$ = exploration parameter (typically $\sqrt{2}$)

### Intuition
- Rarely-pulled arms have **high uncertainty bonus** → get explored
- Frequently-pulled arms have **low uncertainty** → selected based on mean
- Bonus shrinks as $\sqrt{\ln t / N(a)}$ → eventual convergence to best arm

In [ ]:
class UCB1(BanditAlgorithm):
    """
    Upper Confidence Bound (UCB1) Algorithm.
    
    Selects arm with highest UCB = mean + exploration_bonus.
    """
    
    def __init__(self, n_arms: int, c: float = np.sqrt(2)):
        super().__init__(n_arms)
        self.c = c  # Exploration parameter
    
    def select_arm(self) -> int:
        """Select arm with highest UCB."""
        # First, try each arm once
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        # Calculate UCB for each arm
        ucb_values = self.get_ucb_values()
        return np.argmax(ucb_values)
    
    def get_ucb_values(self) -> np.ndarray:
        """Calculate UCB value for each arm."""
        # Avoid division by zero
        counts = np.maximum(self.counts, 1e-10)
        
        # Exploration bonus
        exploration_bonus = self.c * np.sqrt(np.log(self.t + 1) / counts)
        
        return self.values + exploration_bonus
    
    def get_exploration_bonus(self) -> np.ndarray:
        """Get current exploration bonus for each arm."""
        counts = np.maximum(self.counts, 1e-10)
        return self.c * np.sqrt(np.log(self.t + 1) / counts)
    
    def __repr__(self):
        return f"UCB1(c={self.c:.3f})"


class UCB_Tuned(BanditAlgorithm):
    """
    UCB-Tuned: UCB with variance estimation.
    
    Uses observed variance to improve exploration bonus.
    """
    
    def __init__(self, n_arms: int):
        super().__init__(n_arms)
    
    def reset(self):
        super().reset()
        self.sum_squared = np.zeros(self.n_arms)
    
    def update(self, arm: int, reward: float):
        super().update(arm, reward)
        self.sum_squared[arm] += reward ** 2
    
    def select_arm(self) -> int:
        # First, try each arm once
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        ucb_values = self.get_ucb_values()
        return np.argmax(ucb_values)
    
    def get_ucb_values(self) -> np.ndarray:
        """Calculate UCB-Tuned value for each arm."""
        counts = np.maximum(self.counts, 1e-10)
        
        # Estimated variance
        variance = self.sum_squared / counts - self.values ** 2
        variance = np.maximum(variance, 0)  # Ensure non-negative
        
        # V_i bound
        log_t = np.log(self.t + 1)
        v = variance + np.sqrt(2 * log_t / counts)
        
        # UCB-Tuned bonus
        exploration_bonus = np.sqrt(log_t / counts * np.minimum(0.25, v))
        
        return self.values + exploration_bonus
    
    def __repr__(self):
        return "UCB_Tuned()"


# Test UCB
ucb = UCB1(n_arms=5)
print(f"Algorithm: {ucb}")

---
## Part 5: Running Experiments

Let's compare all algorithms on our trading strategy selection problem.

In [ ]:
def run_experiment(env: BanditEnvironment, algorithm: BanditAlgorithm, 
                   n_rounds: int) -> Dict[str, np.ndarray]:
    """
    Run a bandit experiment.
    
    Returns:
        Dictionary with rewards, regrets, and arm selections history
    """
    algorithm.reset()
    
    rewards = np.zeros(n_rounds)
    regrets = np.zeros(n_rounds)
    arms_selected = np.zeros(n_rounds, dtype=int)
    
    cumulative_reward = 0
    cumulative_regret = 0
    
    for t in range(n_rounds):
        # Select and pull arm
        arm = algorithm.select_arm()
        reward = env.pull(arm)
        
        # Update algorithm
        algorithm.update(arm, reward)
        
        # Track metrics
        cumulative_reward += reward
        cumulative_regret += env.get_regret(arm)
        
        rewards[t] = cumulative_reward
        regrets[t] = cumulative_regret
        arms_selected[t] = arm
    
    return {
        'cumulative_rewards': rewards,
        'cumulative_regrets': regrets,
        'arms_selected': arms_selected,
        'final_values': algorithm.values.copy(),
        'final_counts': algorithm.counts.copy()
    }


def run_multiple_experiments(env: BanditEnvironment, algorithm: BanditAlgorithm,
                             n_rounds: int, n_experiments: int) -> Dict[str, np.ndarray]:
    """
    Run multiple experiments and average results.
    """
    all_rewards = np.zeros((n_experiments, n_rounds))
    all_regrets = np.zeros((n_experiments, n_rounds))
    
    for exp in range(n_experiments):
        results = run_experiment(env, algorithm, n_rounds)
        all_rewards[exp] = results['cumulative_rewards']
        all_regrets[exp] = results['cumulative_regrets']
    
    return {
        'mean_rewards': all_rewards.mean(axis=0),
        'std_rewards': all_rewards.std(axis=0),
        'mean_regrets': all_regrets.mean(axis=0),
        'std_regrets': all_regrets.std(axis=0)
    }


# Run experiments
n_rounds = 1000
n_experiments = 100

algorithms = {
    'Epsilon-Greedy (ε=0.1)': EpsilonGreedy(5, epsilon=0.1),
    'Epsilon-Greedy (ε=0.01)': EpsilonGreedy(5, epsilon=0.01),
    'Decaying ε-Greedy': DecayingEpsilonGreedy(5, epsilon_start=1.0, decay=0.01),
    'Thompson Sampling': ThompsonSampling(5),
    'UCB1': UCB1(5),
    'UCB-Tuned': UCB_Tuned(5)
}

results = {}
print(f"Running {n_experiments} experiments with {n_rounds} rounds each...\n")

for name, algo in algorithms.items():
    print(f"Testing {name}...")
    results[name] = run_multiple_experiments(env, algo, n_rounds, n_experiments)

print("\nExperiments complete!")

In [ ]:
# Plot cumulative regret comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = plt.cm.Set2(np.linspace(0, 1, len(algorithms)))

# Cumulative Regret
ax = axes[0]
for (name, res), color in zip(results.items(), colors):
    mean = res['mean_regrets']
    std = res['std_regrets']
    ax.plot(mean, label=name, color=color, linewidth=2)
    ax.fill_between(range(n_rounds), mean - std, mean + std, alpha=0.15, color=color)

ax.set_xlabel('Round')
ax.set_ylabel('Cumulative Regret')
ax.set_title('Cumulative Regret Over Time')
ax.legend(loc='upper left')
ax.set_xlim(0, n_rounds)

# Cumulative Reward
ax = axes[1]
for (name, res), color in zip(results.items(), colors):
    mean = res['mean_rewards']
    ax.plot(mean, label=name, color=color, linewidth=2)

# Add optimal line
optimal = np.arange(1, n_rounds + 1) * env.best_probability
ax.plot(optimal, 'k--', label='Optimal', linewidth=2, alpha=0.7)

ax.set_xlabel('Round')
ax.set_ylabel('Cumulative Reward')
ax.set_title('Cumulative Reward Over Time')
ax.legend(loc='upper left')
ax.set_xlim(0, n_rounds)

plt.tight_layout()
plt.show()

# Print final regrets
print("\n" + "="*60)
print("Final Cumulative Regret (lower is better):")
print("="*60)
sorted_results = sorted(results.items(), key=lambda x: x[1]['mean_regrets'][-1])
for name, res in sorted_results:
    print(f"{name:30s}: {res['mean_regrets'][-1]:.2f} ± {res['std_regrets'][-1]:.2f}")

---
## Part 6: Detailed Analysis of Each Algorithm

In [ ]:
# Run single detailed experiment for visualization
np.random.seed(123)

detailed_results = {}
for name, algo in algorithms.items():
    detailed_results[name] = run_experiment(env, algo, n_rounds=500)

# Plot arm selection frequency over time
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (name, res) in enumerate(detailed_results.items()):
    ax = axes[idx]
    arms = res['arms_selected']
    
    # Calculate running selection frequency
    window = 50
    selection_freq = np.zeros((500 - window + 1, 5))
    
    for t in range(500 - window + 1):
        window_arms = arms[t:t+window]
        for arm in range(5):
            selection_freq[t, arm] = np.sum(window_arms == arm) / window
    
    # Stack plot
    ax.stackplot(range(window-1, 500), selection_freq.T, 
                 labels=strategy_names, alpha=0.8)
    ax.axhline(y=1.0, color='k', linestyle='-', alpha=0.3)
    ax.set_xlabel('Round')
    ax.set_ylabel('Selection Frequency')
    ax.set_title(name)
    ax.set_xlim(window-1, 500)
    ax.set_ylim(0, 1)

# Add legend to last plot
axes[-1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.suptitle('Arm Selection Frequency Over Time (50-round rolling window)', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze final arm counts
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(strategy_names))
width = 0.12
multiplier = 0

for name, res in detailed_results.items():
    counts = res['final_counts']
    percentages = counts / counts.sum() * 100
    offset = width * multiplier
    ax.bar(x + offset, percentages, width, label=name)
    multiplier += 1

ax.set_xlabel('Strategy')
ax.set_ylabel('Selection Percentage (%)')
ax.set_title('Final Arm Selection Distribution')
ax.set_xticks(x + width * 2.5)
ax.set_xticklabels(strategy_names, rotation=15)
ax.legend(loc='upper right')

# Add line for best arm
ax.axvline(x=env.best_arm + width * 2.5, color='green', linestyle='--', 
           linewidth=2, alpha=0.7, label='Best Arm')

plt.tight_layout()
plt.show()

# Print estimated vs true values
print("\n" + "="*80)
print("Estimated Win Rates vs True Win Rates:")
print("="*80)
print(f"{'Strategy':<20} {'True':<10} ", end="")
for name in detailed_results.keys():
    short_name = name[:15]
    print(f"{short_name:<15} ", end="")
print()
print("-"*80)

for i, strat in enumerate(strategy_names):
    print(f"{strat:<20} {true_win_rates[i]:<10.1%} ", end="")
    for name, res in detailed_results.items():
        est = res['final_values'][i]
        print(f"{est:<15.1%} ", end="")
    print()

---
## Part 7: Thompson Sampling Deep Dive

In [ ]:
# Visualize Thompson Sampling belief evolution
from scipy import stats

np.random.seed(42)
ts = ThompsonSampling(n_arms=5)

# Record states at different time points
checkpoints = [10, 50, 100, 500]
states = {}

for t in range(max(checkpoints) + 1):
    arm = ts.select_arm()
    reward = env.pull(arm)
    ts.update(arm, reward)
    
    if t + 1 in checkpoints:
        states[t + 1] = (ts.alpha.copy(), ts.beta.copy())

# Plot belief evolution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

x = np.linspace(0, 1, 200)
colors = plt.cm.tab10(np.linspace(0, 1, 5))

for idx, checkpoint in enumerate(checkpoints):
    ax = axes[idx]
    alpha, beta = states[checkpoint]
    
    for i in range(5):
        y = stats.beta.pdf(x, alpha[i], beta[i])
        ax.plot(x, y, label=strategy_names[i], color=colors[i], linewidth=2)
        ax.fill_between(x, y, alpha=0.15, color=colors[i])
        
        # Mark true probability
        ax.axvline(x=true_win_rates[i], color=colors[i], linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Win Probability')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'After {checkpoint} Rounds')
    ax.set_xlim(0.3, 0.8)
    ax.legend(loc='upper right', fontsize=8)

plt.suptitle('Thompson Sampling: Posterior Evolution\n(Dashed lines = true probabilities)', y=1.02)
plt.tight_layout()
plt.show()

---
## Part 8: UCB Analysis

In [ ]:
# Visualize UCB values over time
np.random.seed(42)
ucb = UCB1(n_arms=5)

# Track UCB components over time
n_track = 200
means_history = np.zeros((n_track, 5))
bonus_history = np.zeros((n_track, 5))
ucb_history = np.zeros((n_track, 5))
arms_history = np.zeros(n_track, dtype=int)

for t in range(n_track):
    arm = ucb.select_arm()
    reward = env.pull(arm)
    ucb.update(arm, reward)
    
    arms_history[t] = arm
    means_history[t] = ucb.values.copy()
    
    if t >= 4:  # After initial exploration
        bonus_history[t] = ucb.get_exploration_bonus()
        ucb_history[t] = ucb.get_ucb_values()

# Plot
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Estimated means
ax = axes[0]
for i in range(5):
    ax.plot(means_history[:, i], label=strategy_names[i], linewidth=2)
    ax.axhline(y=true_win_rates[i], color=f'C{i}', linestyle='--', alpha=0.5)
ax.set_ylabel('Estimated Mean')
ax.set_title('UCB1: Estimated Mean Values')
ax.legend(loc='upper right')
ax.set_ylim(0.3, 0.8)

# Exploration bonus
ax = axes[1]
for i in range(5):
    ax.plot(bonus_history[:, i], label=strategy_names[i], linewidth=2)
ax.set_ylabel('Exploration Bonus')
ax.set_title('UCB1: Exploration Bonus (decays with more pulls)')
ax.legend(loc='upper right')

# UCB values
ax = axes[2]
for i in range(5):
    ax.plot(ucb_history[:, i], label=strategy_names[i], linewidth=2)
ax.set_xlabel('Round')
ax.set_ylabel('UCB Value')
ax.set_title('UCB1: Upper Confidence Bound = Mean + Bonus')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

---
## Part 9: Trading Strategy Selection Application

Let's apply bandits to a more realistic trading scenario with:
- Non-stationary rewards (regime changes)
- Contextual information

In [ ]:
class NonStationaryBandit:
    """
    Non-stationary bandit with regime changes.
    
    Simulates market regime changes where strategy effectiveness varies.
    """
    
    def __init__(self, n_arms: int, regime_length: int = 200):
        self.n_arms = n_arms
        self.regime_length = regime_length
        self.t = 0
        self._generate_regime()
    
    def _generate_regime(self):
        """Generate new regime probabilities."""
        self.true_probabilities = np.random.uniform(0.35, 0.65, self.n_arms)
        self.best_arm = np.argmax(self.true_probabilities)
        self.best_probability = self.true_probabilities[self.best_arm]
    
    def pull(self, arm: int) -> int:
        """Pull arm and potentially change regime."""
        self.t += 1
        
        # Check for regime change
        if self.t % self.regime_length == 0:
            self._generate_regime()
        
        return int(np.random.random() < self.true_probabilities[arm])
    
    def get_regret(self, arm: int) -> float:
        return self.best_probability - self.true_probabilities[arm]


class SlidingWindowUCB(BanditAlgorithm):
    """
    UCB with sliding window for non-stationary environments.
    
    Only considers recent observations.
    """
    
    def __init__(self, n_arms: int, window_size: int = 100, c: float = np.sqrt(2)):
        super().__init__(n_arms)
        self.window_size = window_size
        self.c = c
    
    def reset(self):
        super().reset()
        self.history = []  # List of (arm, reward) tuples
    
    def update(self, arm: int, reward: float):
        self.t += 1
        self.history.append((arm, reward))
        
        # Keep only recent history
        if len(self.history) > self.window_size:
            self.history = self.history[-self.window_size:]
        
        # Recalculate counts and values from window
        self.counts = np.zeros(self.n_arms)
        self.values = np.zeros(self.n_arms)
        
        for a, r in self.history:
            self.counts[a] += 1
        
        for a in range(self.n_arms):
            if self.counts[a] > 0:
                rewards = [r for (arm, r) in self.history if arm == a]
                self.values[a] = np.mean(rewards)
    
    def select_arm(self) -> int:
        # Ensure each arm tried at least once
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        # UCB with window-based counts
        n_window = len(self.history)
        exploration_bonus = self.c * np.sqrt(np.log(n_window + 1) / np.maximum(self.counts, 1))
        ucb_values = self.values + exploration_bonus
        
        return np.argmax(ucb_values)
    
    def __repr__(self):
        return f"SlidingWindowUCB(window={self.window_size})"


# Compare algorithms in non-stationary environment
np.random.seed(42)
ns_env = NonStationaryBandit(n_arms=5, regime_length=200)

ns_algorithms = {
    'Epsilon-Greedy (ε=0.1)': EpsilonGreedy(5, epsilon=0.1),
    'Thompson Sampling': ThompsonSampling(5),
    'UCB1': UCB1(5),
    'Sliding Window UCB': SlidingWindowUCB(5, window_size=100)
}

# Run experiments
n_rounds = 1000
ns_results = {}

print("Running non-stationary experiments...")
for name, algo in ns_algorithms.items():
    # Reset environment for fair comparison
    np.random.seed(42)
    ns_env = NonStationaryBandit(n_arms=5, regime_length=200)
    algo.reset()
    
    rewards = np.zeros(n_rounds)
    regrets = np.zeros(n_rounds)
    cum_reward = 0
    cum_regret = 0
    
    for t in range(n_rounds):
        arm = algo.select_arm()
        reward = ns_env.pull(arm)
        algo.update(arm, reward)
        
        cum_reward += reward
        cum_regret += ns_env.get_regret(arm)
        
        rewards[t] = cum_reward
        regrets[t] = cum_regret
    
    ns_results[name] = {'rewards': rewards, 'regrets': regrets}

print("Done!")

In [ ]:
# Plot non-stationary results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mark regime changes
regime_changes = [200, 400, 600, 800]

# Cumulative Regret
ax = axes[0]
for name, res in ns_results.items():
    ax.plot(res['regrets'], label=name, linewidth=2)

for rc in regime_changes:
    ax.axvline(x=rc, color='red', linestyle='--', alpha=0.5)

ax.set_xlabel('Round')
ax.set_ylabel('Cumulative Regret')
ax.set_title('Non-Stationary Bandits: Cumulative Regret\n(Red lines = regime changes)')
ax.legend(loc='upper left')

# Cumulative Reward
ax = axes[1]
for name, res in ns_results.items():
    ax.plot(res['rewards'], label=name, linewidth=2)

for rc in regime_changes:
    ax.axvline(x=rc, color='red', linestyle='--', alpha=0.5)

ax.set_xlabel('Round')
ax.set_ylabel('Cumulative Reward')
ax.set_title('Non-Stationary Bandits: Cumulative Reward')
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*50)
print("Non-Stationary Environment Results:")
print("="*50)
for name, res in ns_results.items():
    print(f"{name:30s}: Regret = {res['regrets'][-1]:.1f}, Reward = {res['rewards'][-1]:.0f}")

---
## Part 10: Key Takeaways and Summary

### Algorithm Comparison

| Algorithm | Pros | Cons | Best For |
|-----------|------|------|----------|
| **Epsilon-Greedy** | Simple, easy to implement | Fixed exploration rate, no uncertainty | Quick baseline, simple problems |
| **Decaying ε-Greedy** | Reduces exploration over time | Requires tuning decay rate | When confident about convergence |
| **Thompson Sampling** | Bayesian, handles uncertainty naturally | Requires prior specification | General purpose, good default |
| **UCB1** | No hyperparameters, proven bounds | Can over-explore initially | When theoretical guarantees needed |
| **Sliding Window UCB** | Adapts to non-stationarity | Forgets useful information | Changing environments |

### Trading Applications

1. **Strategy Selection**: Use bandits to dynamically allocate capital among strategies
2. **Execution Algorithms**: Test different execution strategies with Thompson Sampling
3. **Market Making**: Adaptive spread setting based on market conditions
4. **Portfolio Rebalancing**: Explore new allocations while exploiting known good ones

### Best Practices

1. **Start with Thompson Sampling** - Good default for most problems
2. **Consider non-stationarity** - Markets change, use sliding windows or discounting
3. **Define rewards carefully** - Use risk-adjusted returns, not just returns
4. **Monitor regret** - Track how much you're leaving on the table
5. **Batch updates** - In practice, update less frequently for stability

In [ ]:
# Summary visualization
print("="*60)
print("WEEK 16 - DAY 2: MULTI-ARMED BANDITS SUMMARY")
print("="*60)

summary = """
📚 KEY CONCEPTS COVERED:

1. EXPLORATION vs EXPLOITATION TRADEOFF
   - Fundamental challenge in sequential decision making
   - Must balance trying new options vs using known good ones

2. EPSILON-GREEDY
   - Explore with probability ε, exploit otherwise
   - Simple but effective baseline
   - Decaying ε reduces exploration over time

3. THOMPSON SAMPLING
   - Bayesian approach with posterior distributions
   - Beta distributions for Bernoulli bandits
   - Naturally balances exploration and exploitation

4. UPPER CONFIDENCE BOUND (UCB)
   - "Optimism in the face of uncertainty"
   - UCB = mean + exploration_bonus
   - Bonus shrinks as arm is pulled more

5. NON-STATIONARY ENVIRONMENTS
   - Real markets change over time
   - Use sliding windows or discounting
   - Constant exploration may be necessary

🎯 TRADING APPLICATIONS:
   • Strategy selection
   • Portfolio allocation
   • A/B testing execution algorithms
   • Adaptive market making

📈 NEXT: Day 3 will cover Markov Decision Processes (MDPs)
"""

print(summary)